In [1]:
import sys
!{sys.executable} -m pip install cvxopt                                          # INSTALL CVXOPT IN JUPYTER
import cvxopt as cpt                                                             # using cvxopt for convex optimization
import numpy as np                                                               # for using numpy
import pandas as pd                                                              # for using pandas
from cvxopt import solvers

Invalid requirement: '#'
Traceback (most recent call last):
  File "C:\Users\AMAN\Anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "C:\Users\AMAN\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1654, in parseString
    raise exc
  File "C:\Users\AMAN\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1644, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "C:\Users\AMAN\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1402, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "C:\Users\AMAN\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 3417, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "C:\Users\AMAN\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1402, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "C:\Use

In [13]:
df = pd.read_csv(r'C:\Users\AMAN\Desktop\machine learning\assignment 2\creditcard.csv') # read csv file as dataframe
df.isnull().values.any()                                                                # check is there any missing vlaue in dataset


df_fraud = df[df['Class'] == 1]                                                    # new dataframe of only fraud data                
df_non_fraud=df[df['Class']==0]                                                    # new dataframe of only non fraud data


for i in range(5):
    df = pd.read_csv(r'C:\Users\AMAN\Desktop\machine learning\assignment 2\creditcard.csv') # read csv file as dataframe
    df.isnull().values.any()                                        # check is there any missing vlaue in dataset
    df_fraud = df[df['Class'] == 1]                                 # new dataframe of only fraud data                
    df_non_fraud=df[df['Class']==0]                                 # new dataframe of only non fraud data
    df_f_1=df_fraud.sample(n=100)                                   # take only 100 datas of fraud cases                               
    df_n_f_1=df_non_fraud.sample(n=100)                             # take only 100 datas of non fraud cases
    df_n_f_1['Class']=-1                                            # converting non fraud class value from 0 to -1 (as we took -1 for other class in SVM)  
    final_df=pd.concat([df_f_1,df_n_f_1])                           # final dataframe that contains 100 fraud and non fraud case each
    DF=final_df.reindex(np.random.permutation(final_df.index))      # shuffling data
    tr_size=int(round(len(DF)*0.8))                                 # for 80 % of train data take size value of 80% data
    otpt=DF['Class']                                                # taking our target feature as different column
    X=DF.drop(['Class'],axis=1)                                     # take X for input values
    x_train=np.asarray(X[:tr_size])                                 # 0 to 80% of inout data as training sample
    x_test=np.asarray(X[tr_size:])                                  # remaining 20% data as test sample
    y_train=np.asarray(otpt[:tr_size])                              # output training data of 80% size of total
    y_test=np.asarray(otpt[tr_size:] )                              # output test data of 20% size
    
    
    q=cpt.matrix(np.ones(tr_size)*-1,tc='d')                        # define q matrix of size 160*1 containig each element=-1                       
    G=cpt.matrix(np.identity(tr_size)*-1,tc='d')                    # G is identity matrix with diagonal element = -1 (160*160)
    h=cpt.matrix(np.zeros(tr_size))                                 # h is marix of zero  (160*1)
    a=cpt.matrix((y_train),tc='d')                                  # a is matrix used to construct A matrix (160*1)
    A=cpt.matrix(np.transpose(a),tc='d')                            # A is matrix of size 1*160
    b=cpt.matrix(np.zeros(1),tc='d')                                # b is matrix of size 1*1 (element=0)


    # To define P some terms are predefined in below statement

    y_m=cpt.matrix(y_train,tc='d')                                  # matrix y_i of size 160*1
    y_m_T=cpt.matrix(np.transpose(y_m),tc='d')                      # matrix y_j is transpose of y_i of size 1*160
    y_m_t=cpt.matrix(np.dot((y_m),(y_m_T)),tc='d')                  # multiplicaiton of y_i*y_j

    x_m=cpt.matrix(np.array(x_train),tc='d')                        # matrix x_i of size 160*30
    x_m_T=cpt.matrix(np.transpose(x_m),tc='d')                      # matrix x_j is transpose of x_i of size 30*160
    x_m_t=cpt.matrix(np.dot((x_m),(x_m_T)),tc='d')                  # multiplicaiton of x_i*x_j

    # Now we can define our P

    P=cpt.matrix(np.dot((y_m_t),(x_m_t)),tc='d')                    # multiplicaiton of y_i*y_j*x_i*x_j
    sol=cpt.solvers.qp(P,q,G,h,A,b,  kktsolver='ldl')               # solve by  optimizing dual equation using cvxopt              
    alpha=cpt.matrix(sol['x'])                                      # to obtain matrix containing value of alpha

    # for w
    t=cpt.matrix(np.transpose(np.multiply(np.array(alpha),np.array(y_m))))  
    W=np.dot(t,x_m)                                                 # W is multiplication of alpha,y_m and x_m

    x_n_c1=np.transpose(df_f_1.drop(['Class'],axis=1))              # define x of term 1 of equation in line 62
    t_1=np.dot(W,x_n_c1)                                            # multiply W and x of term_1 of equatino in line 62 
    t_one=np.min(t_1)                                               # min of term 1
    x_n_c0=np.transpose(df_n_f_1.drop(['Class'],axis=1))            # define x of term 2 of equation in line 62
    t_2=np.dot(W,x_n_c0)                                            # multiply W and x of term_2 of equatino in line 62 
    t_two=np.max(t_2)                                               # min of term 2
    b=-(1/2)*(t_one+t_two)                                          # b=(-1/2)*(min{y=1}W*x+max{y=-1}W*x)
    
    a=np.array(np.dot(x_test,np.transpose(W))+b, dtype=float)       # find value of w*x+b=a
    c=np.array(y_test, dtype=float)                                 # construct array contain corresponding true value of y
    
    l=cpt.matrix(a, tc='d')                                         # matrix for a
    m=cpt.matrix(c, tc='d')                                         # matrix for c
    n=np.multiply(l,m)                                              # element  wise multplication and size=160*1
    count=0
    for i in range(len(n)):                                         # for loop to count error in our model
        if n[i]<1:
            count=count+1
    print(count)
    test_error_1=count
    Accuracy=(1-(test_error_1/len(y_test)))*100                     # accuracy =(1-(error/total))*100
    print(Accuracy)

     pcost       dcost       gap    pres   dres
 0:  6.9821e-12  2.0947e-11  2e+02  1e+01  1e+00
 1:  1.3894e-13  2.7928e-11  2e+00  1e-01  1e-02
 2:  1.3919e-15  2.7928e-11  2e-02  1e-03  1e-04
 3: -4.3477e-16  2.7929e-11  2e-04  1e-05  1e-06
 4: -4.4867e-14  2.8018e-11  2e-06  1e-07  1e-08
 5: -4.4527e-12  3.6828e-11  2e-08  2e-09  1e-10
Optimal solution found.
20
50.0
     pcost       dcost       gap    pres   dres
 0:  3.7192e-10  1.1158e-09  2e+02  1e+01  1e+00
 1:  7.4013e-12  1.4877e-09  2e+00  1e-01  1e-02
 2:  7.4136e-14  1.4877e-09  2e-02  1e-03  1e-04
 3: -2.3680e-14  1.4877e-09  2e-04  1e-05  1e-06
 4: -2.4422e-12  1.4926e-09  2e-06  1e-07  1e-08
 5: -2.4817e-10  1.9699e-09  9e-08  8e-09  6e-10
Optimal solution found.
16
60.0
     pcost       dcost       gap    pres   dres
 0:  4.4952e-11  1.3489e-10  2e+02  1e+01  1e+00
 1:  8.9454e-13  1.7980e-10  2e+00  1e-01  1e-02
 2:  8.9853e-15  1.7981e-10  2e-02  1e-03  1e-04
 3: -3.7294e-16  1.7981e-10  2e-04  1e-05  1e-06
 4: -4.6